In [ ]:
import os
from osgeo import gdal,ogr,osr,gdalnumeric
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
import rasterio
import fiona
import rasterio.mask

Ref: https://stackoverflow.com/questions/69938501/clipping-raster-through-a-shapefile-using-python

In [ ]:
us_counties = gp.read_file(r'\Users\mhardika\Downloads\US_County_Boundaries\US_CountyBndrys.shp')
us_counties = us_counties.to_crs("EPSG:3857")

Function to go through county lists in Arizona

In [ ]:
az_counties = us_counties.loc[us_counties['STATEFP']=='04']

In [ ]:
def create_az_images(county,geometry):
    src_raster_path = '/Users/mhardika/Downloads/az_cultivated/clipped.TIF'
    output_raster_path = '/Users/mhardika/Downloads/az_cultivated/folder/'+ county +'.tif'

    with rasterio.open(src_raster_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, [geometry], crop=True)
        out_meta = src.meta


    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})


    with rasterio.open(output_raster_path, "w", **out_meta) as dest:
        dest.write(out_image)


def convert_to_shape(county):
    ds = gdal.Open('/Users/mhardika/Downloads/az_cultivated/folder/'+county+'.tif')
    ds.RasterCount
    srcband = ds.GetRasterBand(1)
    proj = ds.GetProjection()
    
    shp_proj = osr.SpatialReference()
    shp_proj.ImportFromWkt(proj)
    call_drive = ogr.GetDriverByName('ESRI Shapefile')
    create_shp = call_drive.CreateDataSource('/Users/mhardika/Downloads/az_cultivated/folder/'+county+'_output_shapefile.shp')

    shp_layer = create_shp.CreateLayer('layername',srs=shp_proj)
    new_field = ogr.FieldDefn(str('ID'),ogr.OFTInteger)
    shp_layer.CreateField(new_field)

    gdal.Polygonize(srcband,None,shp_layer,0,[],callback = None)
    create_shp.Destroy()
    ds = None


In [ ]:
az_counties.geometry[0:1]

In [ ]:
for index,row in az_counties.iterrows():
    # print(row['NAME'])
    # print(row['geometry'])
    create_az_images(row['NAME'],row['geometry'])


In [ ]:
for index,row in az_counties.iterrows():
    convert_to_shape(row['NAME'])

In [ ]:
# ds = gdal.Open('/Users/mhardika/Downloads/az_cultivated/folder/county_masked.tif')
ds = gdal.Open('/Users/mhardika/Downloads/az_cultivated/folder/county_masked_some.tif')
ds.RasterCount
srcband = ds.GetRasterBand(1)
srcband.ReadAsArray()

In [ ]:
proj = ds.GetProjection()
shp_proj = osr.SpatialReference()
shp_proj.ImportFromWkt(proj)
call_drive = ogr.GetDriverByName('ESRI Shapefile')
# create_shp = call_drive.CreateDataSource('/Users/mhardika/Downloads/az_cultivated/folder/az_cultivated_output_shapefile.shp')
create_shp = call_drive.CreateDataSource('/Users/mhardika/Downloads/az_cultivated/folder/az_cultivated_some_output_shapefile.shp')

# shp_layer = create_shp.CreateLayer('layername',srs=shp_proj)
shp_layer = create_shp.CreateLayer('layername_some',srs=shp_proj)
new_field = ogr.FieldDefn(str('ID'),ogr.OFTInteger)
shp_layer.CreateField(new_field)

gdal.Polygonize(srcband,None,shp_layer,0,[],callback = None)
create_shp.Destroy()
ds = None

In [ ]:
sample = gp.read_file('/Users/mhardika/Downloads/az_cultivated/folder/az_cultivated_some_output_shapefile.shp')
sample = sample.loc[sample['ID']==121]
sample = sample.to_crs("EPSG:4326")
fig, ax = plt.subplots()

az_counties = az_counties.to_crs("EPSG:4326")
az_counties.plot(ax=ax,color = 'white',edgecolor ='black')
sample.plot(ax=ax,figsize=(50, 50))

In [ ]:
sample.loc[sample['ID']==121]